In [35]:

# downloading dataset
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -O yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -O yellow_tripdata_2023-02.parquet



Downloaded file to yellow_tripdata_2023-01.parquet
Downloaded file to yellow_tripdata_2023-02.parquet


In [36]:
#importing dependencies
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
import numpy as np

In [37]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')


In [38]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [39]:
num_columns = len(df.columns)
print(f'Data for January has {num_columns} columns.')


Data for January has 19 columns.


In [40]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [41]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60


In [42]:
std_dev = df['duration'].std()
print(f"Standard deviation of trip durations in January: {std_dev:.2f}")


Standard deviation of trip durations in January: 42.59


In [43]:
df_filtered = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

In [44]:
fraction_left = len(df_filtered) / len(df)*100
print(f"Fraction of records left after dropping outliers: {fraction_left:.2f}%")


Fraction of records left after dropping outliers: 98.12%


In [45]:
df_filtered.loc[:, 'PULocationID'] = df_filtered['PULocationID'].astype(str)
df_filtered.loc[:, 'DOLocationID'] = df_filtered['DOLocationID'].astype(str)
df_location_ids = df_filtered[['PULocationID', 'DOLocationID']]

In [46]:
data_dict = df_location_ids.to_dict('records')
vec = DictVectorizer()
feature_matrix = vec.fit_transform(data_dict)

In [47]:
num_columns = feature_matrix.shape[1]
print("Number of columns:", num_columns)

Number of columns: 515


In [48]:
model = LinearRegression()
model.fit(feature_matrix, df_filtered['duration'])
y_pred = model.predict(feature_matrix)
rmse = np.sqrt(mean_squared_error(df_filtered['duration'], y_pred))
print(f'The RMSE on the training data is {rmse:.2f}.')


The RMSE on the training data is 7.65.


In [49]:
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds()/60
df_val_filtered = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
df_val_filtered.loc[:, 'PULocationID'] = df_val_filtered['PULocationID'].astype(str)
df_val_filtered.loc[:, 'DOLocationID'] = df_val_filtered['DOLocationID'].astype(str)
df_val_location_ids = df_val_filtered[['PULocationID', 'DOLocationID']]
data_val_dict = df_val_location_ids.to_dict('records')
feature_val_matrix = vec.transform(data_val_dict)

In [50]:
y_pred_val = model.predict(feature_val_matrix)
rmse_val = np.sqrt(mean_squared_error(df_val_filtered['duration'], y_pred_val))
print(f'The RMSE on the validation data is {rmse_val:.2f}.')

The RMSE on the validation data is 7.81.
